In [6]:
import pickle
import networkx as nx
import torch
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx

# Load the knowledge graph from pkl
with open(r"C:\Users\wes\telomerase_comprehensive_knowledge_graph_updated_5.pkl", "rb") as f:
    G = pickle.load(f)

# Ensure G is a NetworkX graph
if not isinstance(G, nx.Graph):
    raise TypeError("The loaded object is not a NetworkX graph")

# Function to get all unique attributes
def get_all_attrs(items):
    all_attrs = set()
    for item in items:
        if isinstance(item, tuple) and len(item) == 3:  # Edge data
            _, _, attrs = item
        elif isinstance(item, tuple) and len(item) == 2:  # Node data
            _, attrs = item
        else:
            continue
        if isinstance(attrs, dict):
            all_attrs.update(attrs.keys())
    return list(all_attrs)

# Function to create a default value for missing attributes
def default_value(attr):
    if attr == 'name':
        return ''
    else:
        return 0  # You might want to adjust this based on your data

# Normalize node attributes
all_node_attrs = get_all_attrs(G.nodes(data=True))
for node in G.nodes():
    for attr in all_node_attrs:
        if attr not in G.nodes[node]:
            G.nodes[node][attr] = default_value(attr)

# Normalize edge attributes
all_edge_attrs = get_all_attrs(G.edges(data=True))
for edge in G.edges():
    for attr in all_edge_attrs:
        if attr not in G.edges[edge]:
            G.edges[edge][attr] = default_value(attr)

# Convert to PyTorch Geometric format
data = from_networkx(G)

print(data)  # Check the loaded graph data

# Print some additional information about the graph
print(f"Number of nodes: {data.num_nodes}")
print(f"Number of edges: {data.num_edges}")
print(f"Node features: {data.num_node_features}")
print(f"Edge features: {data.num_edge_features}")

Data(edge_index=[2, 19818], type=[4331], canonical_smiles=[4331], MolWt=[4331], LogP=[4331], NumHDonors=[4331], NumHAcceptors=[4331], NumRotatableBonds=[4331], NumAromaticRings=[4331], assay_description=[4331], name=[4331], description=[4331], proteinDescription=[4331], molecule_chembl_id=[4331], chromosome=[4331], summary=[4331], aliases=[4331], compound_effectiveness=[4331], entrez_id=[4331], target_chembl_id=[4331], keywords=[4331], gene_type=[4331], proteinExistence=[4331], gpl_id=[4331], target_organism=[4331], sequence=[4331], platform_title=[4331], protein_names=[4331], standard_type=[4331], uniProtkbId=[4331], organism=[4331], uniprot_id=[4331], taxon=[4331], telomerase_activity=[4331], standard_units=[4331], annotationScore=[4331], standard_value=[4331], entryType=[4331], gsm_id=[4331], genes=[4331], pchembl_value=[4331], series_title=[4331], title=[4331], activity_type=[19818], activity_value=[19818], activity_units=[19818], edge_assay_description=[19818], similarity=[19818],

In [8]:

import torch
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data, DataLoader
from sklearn.model_selection import train_test_split
import os
import numpy as np


import torch
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Example for processing node features
node_features = []
le = LabelEncoder()
sc = StandardScaler()

# Convert 'type' to numerical and one-hot encode
type_encoded = le.fit_transform(data.type)
type_one_hot = torch.eye(len(le.classes_))[type_encoded]

# Normalize numerical features
numerical_features = torch.stack([
    torch.tensor(sc.fit_transform(data.MolWt.view(-1, 1)).flatten()),
    torch.tensor(sc.fit_transform(data.LogP.view(-1, 1)).flatten()),
    torch.tensor(sc.fit_transform(data.NumHDonors.view(-1, 1)).flatten()),
    torch.tensor(sc.fit_transform(data.NumHAcceptors.view(-1, 1)).flatten()),
    torch.tensor(sc.fit_transform(data.NumRotatableBonds.view(-1, 1)).flatten()),
    torch.tensor(sc.fit_transform(data.NumAromaticRings.view(-1, 1)).flatten()),
], dim=1)

# Combine features
node_features = torch.cat([type_one_hot, numerical_features], dim=1)
node_features = node_features.float()  # Convert to float32
data.x = node_features
print(f"Processed node features shape: {data.x.shape}")


# Split the data into training and validation sets
num_nodes = data.num_nodes
train_mask, val_mask = train_test_split(range(num_nodes), test_size=0.2, random_state=42)
data.train_mask = torch.zeros(num_nodes, dtype=torch.bool)
data.val_mask = torch.zeros(num_nodes, dtype=torch.bool)
data.train_mask[train_mask] = True
data.val_mask[val_mask] = True

class GCN(nn.Module):
    def __init__(self, num_features, hidden_channels, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x, edge_index):
        x = x.float()  # Ensure input is float32
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = self.dropout(x)
        x = self.conv3(x, edge_index)
        return x

num_classes = 2

model = GCN(num_features=data.num_node_features, hidden_channels=64, num_classes=num_classes)
model = model.float()  # Ensure model parameters are float32
# Assuming a binary classification task, adjust as needed
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, verbose=True)

# Early stopping parameters
patience = 20
best_val_loss = float('inf')
counter = 0

# Create a directory for saving models
os.makedirs('model_checkpoints', exist_ok=True)

# Training loop
model.train()
for epoch in range(1000):  # Increased max epochs
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    
    # Validation
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
        val_acc = (out[data.val_mask].argmax(dim=1) == data.y[data.val_mask]).float().mean()
    
    print(f'Epoch {epoch+1}, Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}, Val Acc: {val_acc.item():.4f}')
    
    # Learning rate scheduler step
    scheduler.step(val_loss)
    
    # Save model periodically (every 50 epochs)
    if (epoch + 1) % 50 == 0:
        torch.save(model.state_dict(), f'model_checkpoints/model_epoch_{epoch+1}.pt')
    
    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'model_checkpoints/best_model.pt')
        counter = 0
    else:
        counter += 1
    
    # Early stopping
    if counter >= patience:
        print(f'Early stopping triggered after {epoch+1} epochs')
        break
    
    model.train()

# Load the best model
model.load_state_dict(torch.save('model_checkpoints/best_model.pt'))

# Final evaluation
model.eval()
with torch.no_grad():
    out = model(data.x, data.edge_index)
    final_val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
    final_val_acc = (out[data.val_mask].argmax(dim=1) == data.y[data.val_mask]).float().mean()

print(f'Final Validation Loss: {final_val_loss.item():.4f}, Final Validation Accuracy: {final_val_acc.item():.4f}')

Processed node features shape: torch.Size([4331, 27])


c:\Users\wes\Miniconda3\envs\gnn\Lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


TypeError: 'NoneType' object is not subscriptable

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATv2Conv, SAGEConv, global_mean_pool, BatchNorm, LayerNorm
from torch_geometric.data import Data, Batch
import logging
import os
import pickle
import networkx as nx
from torch_geometric.utils import from_networkx
from torch_geometric.loader import DataLoader
from sklearn.model_selection import train_test_split
import numpy as np

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class EnhancedTelomeraseGNN(nn.Module):
    def __init__(self, num_node_features, num_edge_features, hidden_channels, num_heads, num_node_types):
        super(EnhancedTelomeraseGNN, self).__init__()
        self.node_embedding = nn.Embedding(num_node_types, hidden_channels)
        self.edge_embedding = nn.Linear(num_edge_features, hidden_channels)
        
        self.conv1 = GATv2Conv(hidden_channels + num_node_features, hidden_channels, heads=num_heads, edge_dim=hidden_channels, concat=False)
        self.norm1 = BatchNorm(hidden_channels)
        self.conv2 = GATv2Conv(hidden_channels, hidden_channels, heads=num_heads, edge_dim=hidden_channels, concat=False)
        self.norm2 = BatchNorm(hidden_channels)
        self.conv3 = SAGEConv(hidden_channels, hidden_channels)
        self.norm3 = BatchNorm(hidden_channels)
        self.layernorm1 = LayerNorm(hidden_channels)
        
        self.lin1 = nn.Linear(hidden_channels, hidden_channels)
        self.lin2 = nn.Linear(hidden_channels, hidden_channels)
        
        self.telomerase_activity_head = nn.Linear(hidden_channels, 1)
        self.compound_effectiveness_head = nn.Linear(hidden_channels, 1)
        self.pchembl_value_head = nn.Linear(hidden_channels, 1)

    def forward(self, x, edge_index, edge_attr, batch, node_type):
        node_embed = self.node_embedding(node_type)
        x = torch.cat([x, node_embed], dim=-1)
        edge_embed = self.edge_embedding(edge_attr)

        x = self.norm1(F.elu(self.conv1(x, edge_index, edge_attr=edge_embed)))
        x = self.norm2(F.elu(self.conv2(x, edge_index, edge_attr=edge_embed)))
        x = self.norm3(F.elu(self.conv3(x, edge_index)))
        x = self.layernorm1(x)

        if batch is None:
            batch = torch.zeros(x.size(0), dtype=torch.long, device=x.device)

        x = global_mean_pool(x, batch)
        x = F.elu(self.lin1(x))
        x = F.dropout(x, p=0.7, training=self.training)
        x = self.lin2(x)

        telomerase_activity = self.telomerase_activity_head(x)
        compound_effectiveness = self.compound_effectiveness_head(x)
        pchembl_value = self.pchembl_value_head(x)

        return telomerase_activity, compound_effectiveness, pchembl_value
    
    
def load_knowledge_graph(file_path):
    with open(file_path, "rb") as f:
        G = pickle.load(f)
    
    if not isinstance(G, nx.Graph):
        raise TypeError("The loaded object is not a NetworkX graph")
    
    return G

from sklearn.preprocessing import LabelEncoder

def prepare_data(G):
    data = from_networkx(G)
    
    # Create node features
    node_features = []
    for _, attrs in G.nodes(data=True):
        features = [
            attrs.get('MolWt', 0),
            attrs.get('LogP', 0),
            attrs.get('NumHDonors', 0),
            attrs.get('NumHAcceptors', 0),
            attrs.get('NumRotatableBonds', 0),
            attrs.get('NumAromaticRings', 0)
        ]
        node_features.append(features)
    data.x = torch.tensor(node_features, dtype=torch.float)
    
    # Create edge features
    edge_features = []
    for _, _, attrs in G.edges(data=True):
        edge_features.append([attrs.get('activity_value', 0)])
    data.edge_attr = torch.tensor(edge_features, dtype=torch.float)
    
    # Create node types
    node_types = [G.nodes[n].get('type', 'unknown') for n in G.nodes()]
    le = LabelEncoder()
    encoded_node_types = le.fit_transform(node_types)
    data.node_type = torch.tensor(encoded_node_types, dtype=torch.long)
    
    # Store the label encoder for future use
    data.type_encoder = le
    
    # Create target variables
    data.telomerase_activity = torch.tensor([G.nodes[n].get('telomerase_activity', 0) for n in G.nodes()]).float().unsqueeze(1)
    data.compound_effectiveness = torch.tensor([G.nodes[n].get('compound_effectiveness', 0) for n in G.nodes()]).float().unsqueeze(1)
    data.pchembl_value = torch.tensor([G.nodes[n].get('pchembl_value', 0) for n in G.nodes()]).float().unsqueeze(1)
    
    logger.info(f"Number of nodes: {data.num_nodes}")
    logger.info(f"Edge index shape: {data.edge_index.shape}")
    logger.info(f"Edge attr shape: {data.edge_attr.shape}")
    logger.info(f"Node feature shape: {data.x.shape}")
    logger.info(f"Node type shape: {data.node_type.shape}")
    
    return data

def train_model(model, data, device, num_epochs=100, lr=0.001):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, verbose=True)
    
    criterion = nn.MSELoss()
    
    # Split the data into train and validation sets
    num_nodes = data.num_nodes
    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    train_mask[:int(0.8 * num_nodes)] = True
    val_mask = ~train_mask
    
    best_val_loss = float('inf')
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        
        logger.info(f"Epoch {epoch+1}: Input shapes:")
        logger.info(f"x: {data.x.shape}")
        logger.info(f"edge_index: {data.edge_index.shape}")
        logger.info(f"edge_attr: {data.edge_attr.shape}")
        logger.info(f"node_type: {data.node_type.shape}")
        
        out = model(data.x, data.edge_index, data.edge_attr, None, data.node_type)
        loss = (criterion(out[0][train_mask], data.telomerase_activity[train_mask]) + 
                criterion(out[1][train_mask], data.compound_effectiveness[train_mask]) + 
                criterion(out[2][train_mask], data.pchembl_value[train_mask]))
        
        loss.backward()
        optimizer.step()
        
        # Validation
        model.eval()
        with torch.no_grad():
            out = model(data.x, data.edge_index, data.edge_attr, None, data.node_type)
            val_loss = (criterion(out[0][val_mask], data.telomerase_activity[val_mask]) + 
                        criterion(out[1][val_mask], data.compound_effectiveness[val_mask]) + 
                        criterion(out[2][val_mask], data.pchembl_value[val_mask]))
        
        logger.info(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')
        
        scheduler.step(val_loss)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': best_val_loss,
                'num_node_features': model.conv1.in_channels - model.node_embedding.embedding_dim,
                'num_edge_features': model.edge_embedding.in_features,
                'hidden_channels': model.conv1.out_channels,
                'num_heads': model.conv1.heads,
                'num_node_types': model.node_embedding.num_embeddings
            }, 'best_model.pth')
    
    return model

def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    logger.info(f"Using device: {device}")

    # Load the knowledge graph
    graph_path = r"C:\Users\wes\telomerase_comprehensive_knowledge_graph_updated_5.pkl"
    G = load_knowledge_graph(graph_path)
    
    # Prepare the data
    data = prepare_data(G)
    data = data.to(device)
    logger.info(f"Data moved to device: {data.x.device}")

    num_node_features = data.x.size(1)
    num_edge_features = data.edge_attr.size(1)
    num_node_types = len(data.type_encoder.classes_)

    model = EnhancedTelomeraseGNN(
        num_node_features=num_node_features,
        num_edge_features=num_edge_features,
        hidden_channels=64,
        num_heads=4,
        num_node_types=num_node_types
    ).to(device)

    trained_model = train_model(model, data, device)
    logger.info("Training completed.")

if __name__ == "__main__":
    main()

2024-09-17 17:45:13,671 - INFO - Using device: cuda


ValueError: Not all nodes contain the same attributes

In [15]:
import torch
from rdkit import Chem

def is_valid_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return mol is not None

# Load the model and tokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("gokceuludogan/WarmMolGenTwo")
model = AutoModelForSeq2SeqLM.from_pretrained("gokceuludogan/WarmMolGenTwo")
def generate_ligand_smiles(protein_sequence, num_samples=5, max_length=512):
    # Prepare the input
    input_text = f"<protein>{protein_sequence}</protein><smiles>"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    # Generate SMILES
    outputs = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=num_samples,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
        repetition_penalty=1.2,
    )

    # Decode and validate SMILES
    generated_smiles = []
    for output in outputs:
        smiles = tokenizer.decode(output, skip_special_tokens=True)
        smiles = smiles.split("<smiles>")[-1].strip()
        if is_valid_smiles(smiles):
            generated_smiles.append(smiles)

    return generated_smiles

# Example usage
protein_sequence = "ISLPKIPFIRYPIKVRQKAWPWHFKIWIYKSAIDHKLASMMGKWTPDQTKPNDTWCFESGPWVERLEKFILNQMITPLTGSENCYWMNHNVSKNTCYTGHLRTSKWEPQEHGIRDFFKFKDNKPFHWDGAAKMQDKRLDQQITRGKEPQNGRETQQMRPGKKILDVCQMSDKLNEPQPWEGDKSVGSQPQWRVHVYPEYHFHCKSHGADLIVPMHHPWCAQMLMGNWYVAPLKSWSFLLMMAQNDTPNWIYKEIMLYCSPRDSFHMNKNFYDSAHQNKENILNLCGNDCDGYKGESVKQCGPDAGVSHVMLPHSKYGKWPRTFAAGQHMWMQEFWINNNQKNENGCRCHYTNLEVSKHDEHQIHVMMGNCATIVTPHVMYLDWASQEMERHRENQYDYVYICSASRDRAGTTKIMWGNGNKALPNLGSKGNLECWGAGIQQVDILWKYKMQNRAPHSLIFGDQLSVWLECCIHNGQGPKQERLSAHEQSFCVFNMRNQVRKKSFHP"
ligand_smiles = generate_ligand_smiles(protein_sequence)

print("Generated valid SMILES:")
for smiles in ligand_smiles:
    print(smiles)


Generated valid SMILES:


[19:15:30] SMILES Parse Error: syntax error while parsing: SL1(AT)YL2LL(-c3DE(C)o3)cALO)c(PS3KL(AV)c3)c2CALO)N1
[19:15:30] SMILES Parse Error: Failed parsing SMILES 'SL1(AT)YL2LL(-c3DE(C)o3)cALO)c(PS3KL(AV)c3)c2CALO)N1' for input: 'SL1(AT)YL2LL(-c3DE(C)o3)cALO)c(PS3KL(AV)c3)c2CALO)N1'
[19:15:30] SMILES Parse Error: syntax error while parsing: IT1KL(c1)N1CALO)c2LL(5NW5LL2C1=O)C1AA3(C)EE(NQLEE3
[19:15:30] SMILES Parse Error: Failed parsing SMILES 'IT1KL(c1)N1CALO)c2LL(5NW5LL2C1=O)C1AA3(C)EE(NQLEE3' for input: 'IT1KL(c1)N1CALO)c2LL(5NW5LL2C1=O)C1AA3(C)EE(NQLEE3'
[19:15:30] SMILES Parse Error: syntax error while parsing: AA[C@H]1N(VG2c1KL2O)GL1LL(C=O)VL1F
[19:15:30] SMILES Parse Error: Failed parsing SMILES 'AA[C@H]1N(VG2c1KL2O)GL1LL(C=O)VL1F' for input: 'AA[C@H]1N(VG2c1KL2O)GL1LL(C=O)VL1F'
[19:15:30] SMILES Parse Error: syntax error while parsing: ATALO)c1LL2[C@H]3O[C@DDCILOAEC@@HDDOAEC@HDDO)AA3(C)c2c(n1DPc1KL(c1)C(F)F
[19:15:30] SMILES Parse Error: Failed parsing SMILES 'ATALO)c1LL2[C@H]

In [20]:
from transformers import EncoderDecoderModel, RobertaTokenizer, pipeline

def is_valid_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return mol is not None



protein_tokenizer = RobertaTokenizer.from_pretrained("gokceuludogan/WarmMolGenTwo")
mol_tokenizer = RobertaTokenizer.from_pretrained("seyonec/PubChem10M_SMILES_BPE_450k")
model = EncoderDecoderModel.from_pretrained("gokceuludogan/WarmMolGenTwo")
inputs = protein_tokenizer("ISLPKIPFIRYPIKVRQKAWPWHFKIWIYKSAIDHKLASMMGKWTPDQTKPNDTWCFESGPWVERLEKFILNQMITPLTGSENCYWMNHNVSKNTCYTGHLRTSKWEPQEHGIRDFFKFKDNKPFHWDGAAKMQDKRLDQQITRGKEPQNGRETQQMRPGKKILDVCQMSDKLNEPQPWEGDKSVGSQPQWRVHVYPEYHFHCKSHGADLIVPMHHPWCAQMLMGNWYVAPLKSWSFLLMMAQNDTPNWIYKEIMLYCSPRDSFHMNKNFYDSAHQNKENILNLCGNDCDGYKGESVKQCGPDAGVSHVMLPHSKYGKWPRTFAAGQHMWMQEFWINNNQKNENGCRCHYTNLEVSKHDEHQIHVMMGNCATIVTPHVMYLDWASQEMERHRENQYDYVYICSASRDRAGTTKIMWGNGNKALPNLGSKGNLECWGAGIQQVDILWKYKMQNRAPHSLIFGDQLSVWLECCIHNGQGPKQERLSAHEQSFCVFNMRNQVRKKSFHP", return_tensors="pt")
outputs = model.generate(**inputs, decoder_start_token_id=mol_tokenizer.bos_token_id, 
                          eos_token_id=mol_tokenizer.eos_token_id, pad_token_id=mol_tokenizer.eos_token_id, 
                          max_length=256, num_return_sequences=5, do_sample=True, top_p=0.95)


generated_smiles = mol_tokenizer.batch_decode(outputs, skip_special_tokens=True)

print("Validity of generated SMILES:")
for smiles in generated_smiles:
    print(f"{smiles}: {is_valid_smiles(smiles)}")

Validity of generated SMILES:
COc1cccc(c1)-c1cc(-c2ccccc2)n2nc(C)cc2n1: True
CC(C)N1CC(CC(O)=O)n2cc(C#N)c(=O)cc2C1=O: True
CCC1(CC)O[C@@]2(C)OC(NC)=C(C(O)O)[C@@H]2O1: True
2C[C@@]1(NC(=O)c1csc(n1)-c1ccc(F)cc1F)C1CC1: False
CCC[C@]1(O)CN(C2CCCC2)C(=O)c2c(O)c(=O)c(cn12)-c1csc(C)n1: True


[19:20:45] SMILES Parse Error: syntax error while parsing: 2C[C@@]1(NC(=O)c1csc(n1)-c1ccc(F)cc1F)C1CC1
[19:20:45] SMILES Parse Error: Failed parsing SMILES '2C[C@@]1(NC(=O)c1csc(n1)-c1ccc(F)cc1F)C1CC1' for input: '2C[C@@]1(NC(=O)c1csc(n1)-c1ccc(F)cc1F)C1CC1'
